# Neuroimaging with Dipy

[Dipy Home Page](http://nipy.org/dipy/)

In this tutorial we'll go through some basic image processing with Dipy. We'll go through how to fit different diffusion models (DTI & CSD) and how to compute tractography in different ways

## Load Image Data
Lets use a subject from the ADNI3 dataset. We'll be reconstrucing tensors and computing tacks, so we'll need need the preprocessed diffusion image, bvecs, and bvals.  Nibabel is a python package that helps with nifti IO as we just saw. 

In [ ]:
import nibabel as nib
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table

# File paths : on grid this is /ifshome/ccorbin/notebooks/data/blah
f_dwi ='./data/002_S_0413_1_DTI_EC_EPI.nii.gz'
f_bvals = './data/002_S_0413_1_bval.txt'
f_bvecs = './data/002_S_0413_1_rotated_bvec_EC.txt'

# Load DWI
img = nib.load(f_dwi)
data = img.get_data()
print(img.affine)
print(data.shape)

# Load Bvals and Bvecs
bvals, bvecs = read_bvals_bvecs(f_bvals, f_bvecs)
print(bvals)
print(bvecs)
gtab = gradient_table(bvals, bvecs, atol=1e-2)

## Bvecs Needs to be Unit Vectors
It would be helpful if Dipy actually converted to unit vectors instead of just spitting out an error when they aren't, but we can do that ourselves with just a few lines of code. Maybe we'll make a pull request!

In [ ]:
import numpy as np
bvecs = np.loadtxt(f_bvecs)
bvecs = [np.divide(b, np.linalg.norm(b)) if np.any(b != 0) else b for b in bvecs]
print(np.asarray(bvecs))

### Fit Tensors to the Data
Now that we've normalized our bvecs we can create a gradient table with the default tolerance level.  Lets use the gradient table and diffusion data to fit tensors


In [ ]:
from dipy.reconst.dti import TensorModel
gtab = gradient_table(bvals, bvecs, atol=1e-2)
print(gtab.bvecs)

# Tensor Model
model = TensorModel(gtab)
ten = model.fit(data)

### Lets look at some TensorFit attributes
We can access eigenvectors for each voxel, FA values, and the diffusion tensor itself

In [ ]:
print("Eigenvectors")
print(ten.evecs.shape)
print(ten.evecs[50][50][50])
print("Fractional Anisotropy")
shape = ten.fa.shape
print(ten.fa[shape[0] //2 ][shape[1] // 2 ][shape[2] // 2])
print("Diffusion Tensor")
print(ten.quadratic_form[50, 40, 40])

### Visualize Tensors
Lets visualize the tensors we just reconstructed on a small slice of the brain

In [ ]:
from dipy.reconst.dti import color_fa
from dipy.data import get_sphere
from dipy.viz import window, actor

fa = ten.fa
RGB = color_fa(fa, ten.evecs)
sphere = get_sphere('symmetric724')
ren = window.Renderer()
evals = ten.evals[:, :, 55:56]
evecs = ten.evecs[:, :, 55:56]
cfa = RGB[:, :, 55:56]
cfa /= cfa.max()
ren.add(actor.tensor_slicer(evals, evecs, scalar_colors=cfa, sphere=sphere, scale=0.3))
window.show(ren)


## Lets Compute Some Tracks With Dipy's EuDX 
EuDX is Dipy's simplest way of computing tracks. Deterministic only, but can propagate with information from a variety of reconstruction techniques.  Here we'll just use tensors. 

In [ ]:
from dipy.tracking.eudx import EuDX
from dipy.reconst.dti import quantize_evecs
from dipy.data import get_sphere
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

sphere = get_sphere('symmetric724')
ind = quantize_evecs(ten.evecs, sphere.vertices)
eu = EuDX(a=ten.fa, ind=ind, seeds=1000000, odf_vertices=sphere.vertices, a_low=.2)
streamlines = Streamlines(eu)
save_trk("eudx.trk", streamlines, img.affine, vox_size=img.header.get_zooms()[0:3], shape=data.shape[0:3])

In [ ]:
# Print the number of streamlines generated
print(len(streamlines))
# Print the first streamline
print(streamlines[0])

### Visualize tracks
We can visualize these streamlines with Dipy's Viz API

In [ ]:
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors

color = line_colors(streamlines)
streamlines_actor = actor.line(streamlines, line_colors(streamlines))
r = window.Renderer()
r.add(streamlines_actor)
window.show(r)

# Look at how streamlines are represented in python
A streamline is represented as a Nx3 numpy array.  A collection of these (typically list) is how we interact with "streamlines" - Dipy now has integrated with [Nibabel's new streamlines api](http://nipy.org/nibabel/reference/nibabel.streamlines.html), so there's actually a streamlines class in Dipy which inherits from Nibabel's ArraySequence class. 

## Streamline Spaces
There are three main different spaces / coordinate systems that you'll find streamlines in.  Understanding these three spaces and how to convert from one space to another is pretty essential when you want to manipulate streamlines (clustering streamines / analyzing microstructural measures along them / connectivity stuff)

### Voxel Space (aka Grid Space)
Streamines in this space go hand in hand with the voxel coordinate system of the image the streamlines were generated from.  A streamline with point (0, 0, 0) would reside at the center fo the first voxel in the image.  In Dipy if you compute tracks with Eudx (or local_tracking which is next) and don't feed in an affine, then your tracks will be returned in this space. 

### Voxmm space (Trackvis space)
This is the space that Trackvis (the visualization program) stores streamlines.  Its defined by the the grid coordinates * voxel size in mm + voxel_size / 2. The plus one is a shift b/c trackvis interprets streamline voxel coordinates such that the origin refers to the top corner of the first voxel To transform streamlines that are in voxel space to Voxmm space you simply multiply each streamline point by its voxel size (with a translation of half the voxel size). 

### RAS+ and mm space (aka World Coordinates)
When you look at the qfrom of a nifti image - its the transformation that takes the image from their voxel coordinates to World coordinates (which is either scanner coordinates or coordinate in relation to some atlas).  Streamlines can also be stored and represented in this space by applying the qform from a nifti (affine) to each streamline point. The origin is going to be near the middle of the brain. 


In [ ]:
# Streamlines in Voxel Space (grid space)
streamlines_voxel = streamlines
print(streamlines_voxel[0])

# Loading Streamlines in Voxmm Space with Nibabel's depricated trackvis library
tracks, hdr = nib.trackvis.read('eudx.trk')
streamlines_voxmm = [trk[0] for trk in tracks]
print(streamlines_voxmm[0])

# Loading streamlines with Nibabel's new streamlines api
tractogram = nib.streamlines.load('eudx.trk')
streamlines_rasmm = tractogram.streamlines
hdr = tractogram.header
print(streamlines_rasmm[0])

### Convert Streamlines in Voxel Coordinates to Trackvis Voxmm Space
We multiply each point by the images voxel size and translate by half the voxel size.  The origin (point [0, 0, 0]) in voxel coordinates refers to the center of the first voxel.  The origin after this conversion refers to the outer corner of the first voxel. Point (0, 0, 0) in voxel coordinates would be (0.5, 0.5, 0.5) in trackvis voxmm coordinates if the voxel sizes were (1, 1, 1)

In [ ]:
from dipy.tracking.streamline import transform_streamlines
from numpy.testing import assert_array_almost_equal

# Construct vox_to_voxmm affine
voxel_size = np.asarray(img.header.get_zooms()[0:3])
vox_to_voxmm = np.eye(4)
vox_to_voxmm[[0, 1, 2], [0, 1, 2]] = voxel_size
vox_to_voxmm[:3, 3] = voxel_size / 2
print(vox_to_voxmm)

# Transform streamlines
streamlines_voxmm_new = transform_streamlines(streamlines_voxel, vox_to_voxmm)

# Assert that transformed streamlines equals the original streamlines (ones in grid space)
assert_array_almost_equal(streamlines_voxmm_new[0], streamlines_voxmm[0], decimal=3)
print(streamlines_voxmm_new[0])
print(streamlines_voxmm[0])

### Convert Streamlines in Voxel Coordinates to RASMM World Coordinates
Here we simply transform each streamline point by the images affine (qfrom) which takes the image from grid coordinates to world coordinates.  Again the qform maps to either scanner coordinates or coordinates in relation to some template ex MNI. 

In [ ]:
from dipy.tracking.streamline import transform_streamlines
from numpy.testing import assert_array_almost_equal

# Get affine and transform streamlines
vox_to_rasmm = img.affine
streamlines_rasmm_new = transform_streamlines(streamlines_voxel, vox_to_rasmm)

# Assert new streamlines are equal to the ones we loaded in rasmm
assert_array_almost_equal(streamlines_rasmm_new[0], streamlines_rasmm[0], decimal=3)
print(streamlines_rasmm_new[0])
print(streamlines_rasmm[0])

## Local Tracking

Lets get more complicated! Dipy's eudx algorithm is its most basic tractography algorithm.  Its been superseded by their local tracking algorithm.  You won't even find eudx in the documentation anymore.  Local tracking lets you define seeding regions, and lets you specifiy whether you want your direction getter to be probabilistic or deterministic. It also lets you define a tissue classifer (which we can use to further constrain the the tracks we generate). 

While we can use more complicated reconstruction techniques with eudx, we'll show how to do them with local tracking. Let's start off by reconstructing our diffusion data with the Constrained Sphereical Deconvolution (CSD) approach.  

### Fit the CSD model to the data and get spherical harmonic coefficients

In [ ]:
from dipy.reconst.csdeconv import auto_response
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel

response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
print(ratio)

csd_model = ConstrainedSphericalDeconvModel(gtab, response)
csd_fit = csd_model.fit(data)
shm_coeff = csd_fit.shm_coeff

### Visualize ODFS

In [ ]:
from dipy.data import get_sphere
from dipy.viz import window, actor

# Get Sphere and use to get odfs from CSDFit object
sphere = get_sphere('symmetric724')
odfs = csd_fit.odf(sphere)
csd_odf_small = odfs[:, :, 50:51]

# Visualization
ren = window.Renderer()
fodf_spheres = actor.odf_slicer(csd_odf_small, sphere=sphere, scale=0.9, norm=False, colormap='plasma')
ren.add(fodf_spheres)
window.show(ren)

### Local Tracking Using A Threshohld Tissue Classifier

For a first shot, lets use local tracking with the typcial threshold tissue classifier.  This means we'll use a scalar map (FA) to tell the algorithm when to stop tracking. To make things a little more interesting, lets add the following specs
* Probabilistic Direction Getter
* Seed From ONLY white matter
* Seed 1 time per voxel, and generate seeds randomly within each voxel. 

In [ ]:
from dipy.tracking.local import ThresholdTissueClassifier
from dipy.tracking.local import LocalTracking
from dipy.direction import ProbabilisticDirectionGetter
from dipy.tracking.utils import random_seeds_from_mask
from dipy.io.streamline import save_trk
from dipy.tracking.streamline import Streamlines
import numpy as np

# Define the probabilistic direction getter
pg = ProbabilisticDirectionGetter.from_shcoeff(shm_coeff, max_angle=40., sphere=sphere)

# We have a white matter pve map from FSL's FAST Algorithm
f_white_matter_pve = './data/002_S_0413_1_T1_masked_N3_2Colins_6p_pve_2.nii.gz'
wm_pve_img = nib.load(f_white_matter_pve)
wm_pve_data = wm_pve_img.get_data()

# We only want values over 0.9, this becomes our seeding mask
seed_mask = np.ones(wm_pve_data.shape)
seed_mask[wm_pve_data < 0.9] = 0

# Lets generate 1 seed per voxel randomly distributed in the voxel from the wm mask
seeds = random_seeds_from_mask(seed_mask, seeds_count=1, affine=np.eye(4))

# Define the tissue classifier
fa = ten.fa
classifier = ThresholdTissueClassifier(fa, .20)

# Compute tracks
streamlines = LocalTracking(pg, classifier, seeds, np.eye(4), step_size=0.5)
streamlines = Streamlines(streamlines)
print(streamlines[0])

# Save tracks
save_trk("Probabilistic_WMseeding_FAthresh.trk", streamlines, img.affine, vox_size=img.header.get_zooms()[0:3], shape=data.shape[0:3])

### Visualize Probabilistic Tracks with FA threshold
Again we can use dipy's viz api to look at what we just generated

In [ ]:
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors

color = line_colors(streamlines)
streamlines_actor = actor.line(streamlines, line_colors(streamlines))
r = window.Renderer()
r.add(streamlines_actor)
window.show(r)

### Local Tracking Using A Binary Tissue Classifier

Next lets use a binary tissue classifier instead of a scalar map. 

* Seed from only white matter like before
* But lets make a deterministic direction getter. 

In [ ]:
from dipy.direction import DeterministicMaximumDirectionGetter
from dipy.tracking.local import BinaryTissueClassifier

# Make white mattter mask
f_white_matter_pve = './data/002_S_0413_1_T1_masked_N3_2Colins_6p_pve_2.nii.gz'
wm_pve_img = nib.load(f_white_matter_pve)
wm_pve_data = wm_pve_img.get_data()
white_matter_mask = np.ones(wm_pve_data.shape)
white_matter_mask[wm_pve_data < 0.3] = 0

# Define direction getter with spherical harmonics
dg = DeterministicMaximumDirectionGetter.from_shcoeff(shm_coeff, max_angle=40., sphere=sphere)

# Define the tissue classifier
binary_classifier = BinaryTissueClassifier(white_matter_mask)

# Compute streamlines
streamlines = LocalTracking(dg, binary_classifier, seeds, np.eye(4), step_size=0.5)
streamlines = Streamlines(streamlines)
save_trk("Determinsitc_WMseeding_Binarythresh.trk", streamlines, img.affine, vox_size=img.header.get_zooms()[0:3],  shape=data.shape[0:3])

### Visualization Again...
Lets use dipy's api ... again

In [ ]:
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors

color = line_colors(streamlines)
streamlines_actor = actor.line(streamlines, line_colors(streamlines))
r = window.Renderer()
r.add(streamlines_actor)
window.show(r)

### Local Tracking Using ACT (Anatomically Constrained Tractography)
Now lets get really fancy.  We're going to compute Anatomically Constrained Tractography by using all three PVE maps output by FSL's Fast algorithm.  Lets also get fancy with our seeding mask.  We'll use the following specs
* Deterministic Direction Getter
* Seed from the interface between the gray and white matter
* Seed 1 times per voxel, again randomly distributed within each voxel. 


In [ ]:
import matplotlib.pyplot as plt

# Load our pve data - we've already loaded the white matter
f_wm_pve = './data/002_S_0413_1_T1_masked_N3_2Colins_6p_pve_2.nii.gz'
f_gm_pve = './data/002_S_0413_1_T1_masked_N3_2Colins_6p_pve_1.nii.gz'
f_csf_pve = './data/002_S_0413_1_T1_masked_N3_2Colins_6p_pve_0.nii.gz'
wm_pve_img = nib.load(f_wm_pve)
gm_pve_img = nib.load(f_gm_pve)
csf_pve_img = nib.load(f_csf_pve)
wm_pve_data = wm_pve_img.get_data()
gm_pve_data = gm_pve_img.get_data()
csf_pve_data = csf_pve_img.get_data()

# Show them side by side
fig = plt.figure(figsize=(16,16))

plt.subplot(131)
plt.xticks([])
plt.yticks([])
plt.title("White Matter")
plt.imshow(wm_pve_data[:, :, wm_pve_data.shape[2] // 2].T, cmap='gray', origin='lower',
           interpolation='nearest')
fig.tight_layout()

plt.subplot(132)
plt.xticks([])
plt.yticks([])
plt.title('Gray Matter')
plt.imshow(gm_pve_data[:, :, gm_pve_data.shape[2] // 2].T, cmap='gray', origin='lower',
           interpolation='nearest')
fig.tight_layout()

plt.subplot(133)
plt.xticks([])
plt.yticks([])
plt.title('CSF')
plt.imshow(csf_pve_data[:, :, csf_pve_data.shape[2] // 2].T, cmap='gray', origin='lower',
           interpolation='nearest')
fig.tight_layout()

plt.show()

### Generate Seeding Mask in White Gray Matter Interface

In [ ]:
# Create our seeding mask
seedmask = np.zeros(wm_pve_data.shape)
seedmask[(wm_pve_data > 0.05) & (gm_pve_data > 0.05)] = 1

# Show Seeding mask
fig = plt.figure(figsize=(8,8))
plt.xticks([])
plt.yticks([])
plt.title("Interface Between White and Gray Matter")
plt.imshow(seedmask[:, :, seedmask.shape[2] // 2].T, cmap='gray', origin='lower',
           interpolation='nearest')
fig.tight_layout()
plt.show()

### Instantiate Direction Getter, Tissue Classifier, and Compute Tracks

In [ ]:
from dipy.tracking.local import ActTissueClassifier
from dipy.direction import DeterministicMaximumDirectionGetter
from dipy.tracking.local import LocalTracking
from dipy.tracking.utils import random_seeds_from_mask
from dipy.data import default_sphere
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# Instantiate Direction Getter
dg = DeterministicMaximumDirectionGetter.from_shcoeff(shm_coeff, max_angle=40., sphere=default_sphere)

# Get Seeds
seeds = random_seeds_from_mask(seedmask, seeds_count=1, affine=np.eye(4))

# Generate Include and Exclude Maps for ACT Classifier
include_map = gm_pve_data
exclude_map = csf_pve_data

# Add background so that tracks can exit the brain through cst
background = np.ones(wm_pve_data.shape)
background[(gm_pve_data +
            wm_pve_data +
            csf_pve_data) > 0] = 0
include_map[background > 0] = 1

# Instantiate classifier
act_classifier = ActTissueClassifier(include_map, exclude_map)

# Compute Tracks
streamlines = LocalTracking(dg, act_classifier, seeds, np.eye(4), step_size=0.5, return_all=False)
streamlines = Streamlines(streamlines)

# Save Tracks
save_trk("Deterministic_ACT_Classifier.trk", streamlines, img.affine, vox_size=img.header.get_zooms()[0:3],  shape=data.shape[0:3])

### And finally... we visualize again

In [ ]:
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors

color = line_colors(streamlines)
streamlines_actor = actor.line(streamlines, line_colors(streamlines))
r = window.Renderer()
r.add(streamlines_actor)
window.show(r)